# Candlestick body & pattern & return relationship

In [19]:
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import os
from pathlib import Path
import pandas_ta as ta
import pspriceaction.price_action as pa
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [20]:
data = []
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
algo_dir = current_dir.parent.parent
transform_csv_file = str(current_dir) + '/VN30F1M_5minutes_pattern.csv'
if os.path.isfile(transform_csv_file):
    data = pd.read_csv(transform_csv_file, index_col='Date', parse_dates=True)
    data['model'] = data['model'].fillna(value='')
else:
    csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
    is_file = os.path.isfile(csv_file)
    if is_file:
        dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    else:
        dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)
    data = dataset.copy()
    data = pa.pattern_modeling(data)
    data.to_csv(transform_csv_file)

In [21]:
### Ignore this year data => tranh over fitting
data = data[data.index < '2024-01-01 00:00:00']

## Calculating RSI

In [22]:
data["RSI"] = ta.rsi(data["Close"], length=14)

### Calculating return

In [23]:
from cal_return_4_patterns import cal_return

In [24]:
%%time
data = cal_return(data)

CPU times: user 17.3 s, sys: 11.4 ms, total: 17.3 s
Wall time: 17.3 s


### Return Analytics

In [25]:
has_return = data[data['return'] != '']
#----
long_return = has_return[has_return['signal'] == 'long']
short_return = has_return[has_return['signal'] == 'short']
#----
negative_return = has_return[has_return['return'] < 0]
positive_return = has_return[has_return['return'] > 0]
#----
short_negative_return = short_return[short_return['return'] < 0]
short_positive_return = short_return[short_return['return'] > 0]
long_negative_return = long_return[long_return['return'] < 0]
long_positive_return = long_return[long_return['return'] > 0]

## Patterns model & RSI & return relationship

### LONG signal

In [26]:
long_return['RSI'].describe()

count    5755.000000
mean       59.707912
std        13.256837
min        11.767797
25%        51.080894
50%        60.219411
75%        69.014720
max        95.489359
Name: RSI, dtype: float64

In [27]:
long_return['RSI_group'] = pd.qcut(long_return['RSI'], 20)
a = long_return.groupby(['RSI_group'])[['return']].mean()
b = a[['return']].rename(columns={'return': 'mean'})
c = long_return.groupby(['RSI_group'])[['return']].count()
d = c[['return']].rename(columns={'return': 'count'})
e = pd.merge(b, d, left_index=True, right_index=True, how="left")
e

,mean,count
RSI_group,,
"(11.767000000000001, 36.779]",-0.247917,288
"(36.779, 42.333]",0.407986,288
"(42.333, 46.003]",0.438194,288
"(46.003, 48.785]",-0.012544,287
"(48.785, 51.081]",0.403125,288
"(51.081, 53.14]",0.490972,288
"(53.14, 55.061]",0.147387,287
"(55.061, 56.736]",-0.257986,288
"(56.736, 58.458]",-0.105903,288


### SHORT signal

In [28]:
short_return['RSI_group'] = pd.qcut(short_return['RSI'], 20)
f = short_return.groupby(['RSI_group'])[['return']].mean()
g = f[['return']].rename(columns={'return': 'mean'})
h = short_return.groupby(['RSI_group'])[['return']].count()
i = h[['return']].rename(columns={'return': 'count'})
j = pd.merge(g, i, left_index=True, right_index=True, how="left")
j

,mean,count
RSI_group,,
"(2.6510000000000002, 19.39]",1.883529,255
"(19.39, 23.99]",0.966142,254
"(23.99, 26.872]",1.14549,255
"(26.872, 29.333]",0.83189,254
"(29.333, 31.462]",0.63451,255
"(31.462, 33.503]",0.504331,254
"(33.503, 35.356]",0.103529,255
"(35.356, 37.07]",0.288189,254
"(37.07, 38.787]",0.721569,255
